In [100]:
%matplotlib inline
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [5]:
spark= SparkSession.builder.appName('Basics').getOrCreate()

In [27]:
#Fix the CSV file with comma within fields
#Adding "option('maxColumns','13').option('escape','"')"

df = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').option('escape','"').load('dataset/yelp_business.csv')
df.show(10)

+--------------------+--------------------+------------+--------------------+--------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+
|         business_id|                name|neighborhood|             address|          city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          categories|
+--------------------+--------------------+------------+--------------------+--------------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+
|FYWN1wneV18bWNgQj...|  "Dental by Design"|        null|"4855 E Warner Rd...|     Ahwatukee|   AZ|      85044|   33.3306902|  -111.9785992|  4.0|          22|      1|Dentists;General ...|
|He-G7vWjzVUysIKrf...|"Stephen Szabo Sa...|        null|"3101 Washington Rd"|      McMurray|   PA|      15317|   40.2916853|   -80.1048999|  3.0|          11|      1|Hair Stylists;Hai...|
|KQPW8lFf1y5BT2Mxi...|"Western Motor Ve...|        null|"602

In [79]:
count

NameError: name 'count' is not defined

In [84]:
df.groupby('state').count().sort('count',ascending=False).show(10)



+-----+-----+
|state|count|
+-----+-----+
|   AZ|52214|
|   NV|33086|
|   ON|30208|
|   NC|12956|
|   OH|12609|
|   PA|10109|
|   QC| 8169|
|   WI| 4754|
|  EDH| 3795|
|   BW| 3118|
+-----+-----+
only showing top 10 rows



Select "Las Vegas, NV" as the city for analyzing

In [30]:
df_lv = df.filter('state ="NV"').filter('city="Las Vegas"')

Filter out business categories that are not restaurant

In [41]:
df_restaurant = df_lv.filter(df_lv.categories.rlike(r'[Rr]estaurants'))
df_restaurant.show(20)

+--------------------+--------------------+-------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+
|         business_id|                name| neighborhood|             address|     city|state|postal_code|     latitude|     longitude|stars|review_count|is_open|          categories|
+--------------------+--------------------+-------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+-------+--------------------+
|Pd52CjgyEU3Rb8co6...|"Flight Deck Bar ...|    Southeast|"6730 S Las Vegas...|Las Vegas|   NV|      89119|   36.0669136|  -115.1708484|  4.0|          13|      1|Nightlife;Bars;Ba...|
|4srfPk1s8nlm1Yusy...|            "Subway"|    Southeast|"6889 S Eastern A...|Las Vegas|   NV|      89119|   36.0646523|  -115.1189541|  2.5|           6|      1|Fast Food;Restaur...|
|n7V4cD-KqqE3OXk0i...|         "GameWorks"|    Southeast|"6587 Las Vegas B...|La

Distribution of restaurant rating in Las Vagas

In [48]:
df_restaurant.groupby('stars').count().sort('stars', ascending=False).show()

+-----+-----+
|stars|count|
+-----+-----+
|  5.0|  178|
|  4.5|  718|
|  4.0| 1474|
|  3.5| 1411|
|  3.0|  985|
|  2.5|  621|
|  2.0|  356|
|  1.5|  136|
|  1.0|   20|
+-----+-----+



Distribution of restaurant rating less than 3.5 stars in Las Vagas

In [50]:
df_restaurant.filter("stars < 3.5").groupby('stars').count().sort('stars', ascending=False).show()

+-----+-----+
|stars|count|
+-----+-----+
|  3.0|  985|
|  2.5|  621|
|  2.0|  356|
|  1.5|  136|
|  1.0|   20|
+-----+-----+



Filter out data which higher than 3 stars

In [87]:
df_restaurant = df_restaurant.filter("stars < 3.5")

Join the business data with the review data

In [112]:
df_reviews = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').option('escape','"').load('dataset/yelp_review.csv')
#rename the star to review_star to avoid duplicate column name when joining
df_reviews = df_reviews.select( '*', col('stars').alias('review_stars')).drop('stars')

In [113]:
df_join = df_reviews.join(df_restaurant, "business_id")
df_join.show(10)

+--------------------+--------------------+--------------------+----------+--------------------+------+-----+----+------------+----------------+------------+--------------------+---------+-----+-----------+---------+-----------+-----+------------+-------+-------------------+
|         business_id|           review_id|             user_id|      date|                text|useful|funny|cool|review_stars|            name|neighborhood|             address|     city|state|postal_code| latitude|  longitude|stars|review_count|is_open|         categories|
+--------------------+--------------------+--------------------+----------+--------------------+------+-----+----+------------+----------------+------------+--------------------+---------+-----+-----------+---------+-----------+-----+------------+-------+-------------------+
|-cxD1NimFldATDUsN...|LWrocEdhy-S0uAkxs...|5aP7I_IRRct_7Jdp-...|2014-03-02|Reading all these...|     1|    0|   0|           4|"Coasta Cantina"|        null|"4500 W Tropica

Only keep business_id, stars, text, stars, city columns

In [116]:
df_new = df_join.select('business_id','city', 'name','stars','review_stars', 'text')
df_new.show(10)

+--------------------+---------+----------------+-----+------------+--------------------+
|         business_id|     city|            name|stars|review_stars|                text|
+--------------------+---------+----------------+-----+------------+--------------------+
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           4|Reading all these...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           3|Located in the Or...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           4|I love this place...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           1|The big special i...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           2|The food was only...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           4|Try the ceviche a...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           3|The fajita specia...|
|-cxD1NimFldATDUsN...|Las Vegas|"Coasta Cantina"|  2.0|           2|Customer service ...|
|-cxD1NimF

In [117]:
df_new.count()

159580